In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import sys, os, inspect
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot 
from basic_snapshot import basic_snapshot, VICO_loc, FREQS
import plot as plot
import slices as slices
from ionfrac import VMIN, VMAX
# import flux

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.colors import TwoSlopeNorm

from PIL import Image, ImageDraw, ImageFont
from IPython.display import Image as DisplayImage

In [ ]:
nums = np.array([4, 9, 21, 39, 54, 68, 94])
years = np.array(['4,000 yrs', '9,000 yrs', '21,000 yrs', '39,000 yrs', '54,000 yrs', '68,000 yrs', '94,000 yrs'])
masss = np.array([r'1.4 M$_\odot$', r'2 M$_\odot$', r'4 M$_\odot$',
                   r'8 M$_\odot$', r'12 M$_\odot$', r'16 M$_\odot$',
                   r'24 M$_\odot$'])
freqs = FREQS

r_kpc = 1
heights_and_scales = np.load(VICO_loc+'/Data/heights_and_scales.npz')
scales = heights_and_scales['scales'] # AU
heights = heights_and_scales['heights'] # AU  
snaps = np.empty_like(nums, dtype=snapshot)
for ii, num in enumerate(nums):
    snaps[ii] = snapshot(snap=num, name = ('Snap%03d_n' % num), read_zeusmp = False)
    snaps[ii].load_shock_variables()
    snaps[ii].load_fluxes()

### this file specific

In [ ]:
freq_strings=np.array(['.01', '.05', '0.1', '0.5', '1.0', 
                       '5.3', '23', '43', '100', '230']) 

saveloc = VICO_loc+'/pillowfiles/intensities_12msun_fig'
vmin_intensity_noratio, vmax_intensity_noratio = -4, 3 
vmin_intensity_ratio, vmax_intensity_ratio = -4, -1

s=4

In [ ]:
def intensity_pcolormesh(Snap, year, scale, f, vmin, vmax, cmap='CMRmap',
                        saveloc=False, show=False, show_cbar= False, show_xlabels=True, show_ylabels=True, cbar_pad=.2,
                             vertical_cbar=False):
    nu=freqs[f]
    Snap.load_intensity_variables(nu)
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')

    cont = ax.pcolormesh(Snap.X2_1v, Snap.X1_2v, np.log10(np.rot90(Snap.InuA[:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('$x$ (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('$z$ (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
        
    if(show_cbar):
        if(vertical_cbar):
            cbar = fig.colorbar(cont, orientation = 'vertical', 
                                fraction=0.038, pad=cbar_pad)
            cbar.set_label(label = (r'log($I_\nu$ / [mJy/as])'),
                          fontsize = 14)
            cbar.ax.tick_params(rotation=45, labelsize = 13)
        else:
            cbar = fig.colorbar(cont, orientation = 'horizontal', 
                                fraction=0.038, pad=cbar_pad)
            cbar.set_label(label = (r'log($I_\nu$ / [mJy/as])'),
                          fontsize=14)
            cbar.ax.tick_params(rotation=45, labelsize=13)
    ax.text(.02, .29, (freq_strings[f]+'GHz'), ha='left', va='top', 
            transform=ax.transAxes, fontsize = 20, color='white', weight='bold')
    ax.text(.02, .3, ('\n'+year), ha='left', va='top', 
            transform=ax.transAxes, fontsize = 28, color='white', weight='bold')
    if (saveloc != False): 
        filename = saveloc+'/'+Snap.name+'_'+str(nu/10**9)+'GHzIntensity_'+str(scale)+'AU.png' 
        fig.savefig(filename, bbox_inches='tight')
        plt.close(fig)
        return(filename)
        
def intensity_const_pcolormesh(Snap, year, const, scale, f, vmin, vmax, cmap='CMRmap',
                        saveloc=False, show=False, show_cbar= False, show_xlabels=True, show_ylabels=True, cbar_pad=.2,
                             vertical_cbar=False):
    nu=freqs[f]
    Snap.load_intensity_variables(nu)
    Snap.load_intensity_variables_const(nu=nu, const=const)
    
    Snap.X1_2v, Snap.X2_1v = np.meshgrid(Snap.x1, Snap.x2)
    fig = plt.figure(figsize = (4,4))
    fig.tight_layout()
    ax = plt.subplot()
    ax.set_aspect(1)
    ax.set_xlim(-scale/2, scale/2)
    ax.set_ylim(0, scale)
    ax.set_facecolor('black')
    
    cont = ax.pcolormesh(Snap.X2_1v, Snap.X1_2v, np.log10(np.rot90(Snap.InuA_const[:,:])), 
                               vmin=vmin, vmax=vmax, cmap = cmap, shading='nearest')
    if(show_xlabels):
        ax.set_xticks([-scale*.25, 0, scale*.25])
        ax.set_xlabel('$x$ (au)', fontsize=28)
        ax.tick_params(axis='x', labelsize=20)
    else:
        ax.tick_params(bottom=False)
        ax.set_xticks([])
    if(show_ylabels):
        ax.set_yticks([scale*.25,scale*.5,scale*.75])
        ax.set_ylabel('$z$ (au)', fontsize=28)
        ax.tick_params(axis='y', labelsize=20)
    else:
        ax.tick_params(left=False)
        ax.set_yticks([])
        
    if(show_cbar):
        if(vertical_cbar):
            cbar = fig.colorbar(cont, orientation = 'vertical', 
                                fraction=0.038, pad=cbar_pad)
            cbar.set_label(label = (r'log($I_\nu$ / [mJy/as])'),
                          fontsize = 14)
            cbar.ax.tick_params(rotation=45, labelsize = 13)
        else:
            cbar = fig.colorbar(cont, orientation = 'horizontal', 
                                fraction=0.038, pad=cbar_pad)
            cbar.set_label(label = (r'log($I_\nu$ / [mJy/as])'),
                          fontsize=14)
            cbar.ax.tick_params(rotation=45, labelsize=13)
    ax.text(.02, .29, (freq_strings[f]+'GHz'), ha='left', va='top', 
            transform=ax.transAxes, fontsize = 20, color='white', weight='bold')
    ax.text(.02, .3, ('\n'+year), ha='left', va='top', 
            transform=ax.transAxes, fontsize = 28, color='white', weight='bold')
    if ((saveloc) and (const)): 
        filename = saveloc+'/'+Snap.name+'_'+str(nu/10**9)+'GHzIntensity_const_'+str(scale)+'AU.png'             
        fig.savefig(filename, bbox_inches='tight')
        plt.close(fig)
        return filename
    elif ((saveloc)):  #  and (!const)
        filename = saveloc+'/'+Snap.name+'_'+str(nu/10**9)+'GHzIntensity_ratio_'+str(scale)+'AU.png'             
        fig.savefig(filename, bbox_inches='tight')
        plt.close(fig)
        return filename
    

In [ ]:
# def make_Inu12msun_allimage(imgfiles_arr, allimagename, scale, ratio=False):
#     nrows = len(imgfiles_arr) # should be 2
#     ncols = len(imgfiles_arr[0]) # should be 5 for 
#     side=300
#     header=0
#     if(scale==25000):
#         leftax = 150
#     else:
#         leftax=134
#     bottomax = 82
#     allimage=Image.new("RGBA", (ncols*side + leftax, 
#                                 nrows*side + bottomax + header))

#     for r in range(nrows):
#         for c in range(ncols):
#             img = Image.open(imgfiles_arr[r,c])
#             width, height = img.size
# #             print(width,height)
#             if(c==0): # y axis
#                 new_width=int(side*((width-6.25)/(231-6.25-7.25)))
#                 left=0
#                 new_x=0
#                 right=width-6.25
#             else: 
#                 new_width=side
#                 left=7.25
#                 new_x=c*side+(leftax)
#                 right=width-6.25

#             if(r==nrows-1): # x axis
#                 new_height=int(side*(height-6.75)/(231-6.75-6.75))
#                 bottom=height
#             else: 
#                 new_height=side
#                 bottom=height-6.75
#             top=7
#             new_y = r*side + header

#             img = img.crop((left, top, right, bottom))
#             img = img.resize((new_width,new_height))
#             allimage.paste(img, ((new_x, new_y)))
#     display(allimage)
#     if(ratio):
#         imcbar = Image.open(VICO_loc+'/PaperImages/intensities_12msun/cbar_vertical_ratio.png')
#     else:
#         imcbar = Image.open(VICO_loc+'/PaperImages/intensities_12msun/cbar_vertical_noratio.png')
#     imcbar = imcbar.resize((2*imcbar.width, 2*imcbar.height))
#     imnew = Image.new('RGBA', (allimage.width+imcbar.width, allimage.height),
#                  color='white')
#     imnew.paste(allimage, (0,0))
#     imnew.paste(imcbar, (allimage.width, int(allimage.height/2 - imcbar.height/2)))
    
#     display(imnew)
#     imnew.save(allimagename)
    

# Colorbars

## 1 cbar Flooded

In [ ]:
s=4
f=1
cname = intensity_pcolormesh(snaps[s], years[s], scale=25000, f=f,
                          vmin=vmin_intensity_noratio, vmax=vmax_intensity_noratio,
                          show_xlabels=False, show_ylabels=False, 
                          show_cbar=True, vertical_cbar=True,
                                    saveloc=saveloc, cbar_pad=0.05)
cbimg = plot.save_cbar(cname, savename=saveloc+'/cbar_vertical_flooded.png', orientation='vertical')
display(cbimg)

## 2 cbar Cooling

In [ ]:
cname = intensity_const_pcolormesh(snaps[s], years[s], False, 4000, f=f,
                          vmin=vmin_intensity_ratio, vmax=vmax_intensity_ratio,
                          show_xlabels=False, show_ylabels=False, 
                          show_cbar=True, vertical_cbar=True,
                                    saveloc=saveloc, cbar_pad=0.13)
cbimg = plot.save_cbar(cname, savename=saveloc+'/cbar_vertical_cooling.png', orientation='vertical')
display(cbimg)

# 1 Flooded 25000 au 

In [ ]:
s=4
saveloc = VICO_loc+'/pillowfiles/intensities_12msun_fig'

nrows = 2
ncols = 5
scale = 25000
Inu12msun_files_25000au = np.empty((nrows, ncols), dtype=object)

r = -1
c = 0
for f in range(len(freqs)):
    if(f==0 or f==5):
        show_ylabels=True
        r += 1
        c = 0
    else: show_ylabels=False 
    if(f<5): 
        show_xlabels=False
    else: show_xlabels=True
    filename = intensity_pcolormesh(snaps[s], masss[s], scale, 
                              f, vmin_intensity_noratio, 
                              vmax_intensity_noratio,
                              saveloc=saveloc,
                              show_xlabels = show_xlabels, 
                              show_ylabels = show_ylabels, 
                              show_cbar=False)
    Inu12msun_files_25000au[r,c] = filename
    # print(r,c)
    c+=1

In [ ]:
cname = intensity_pcolormesh(snaps[s], years[s], scale=25000, f=f,
                          vmin=vmin_intensity_noratio, vmax=vmax_intensity_noratio,
                          show_xlabels=False, show_ylabels=False, 
                          show_cbar=True, vertical_cbar=True,
                                    saveloc=saveloc+'/cbars', cbar_pad=0.05)
cbimg = plot.save_cbar(cname, savename=saveloc+'/cbar_vertical_flooded.png', orientation='vertical')
# display(cbimg)
cbimg = cbimg.resize((int(1.5*cbimg.width), int(1.5*cbimg.height)))

allimage, details = plot.pil_image(Inu12msun_files_25000au)
allimage = plot.img_concat_right(allimage, cbimg) #, yy=cbar_yy)
display(allimage)
allimage.save(VICO_loc+'/figures/intensity_maps/Inu12Msun_flooded_25000au.png')

# 2 Flooded 4000 au 

In [ ]:
s=4
saveloc = VICO_loc+'/pillowfiles/intensities_12msun_fig'

nrows = 2
ncols = 5
scale = 4000
Inu12msun_files_4000au = np.empty((nrows, ncols), dtype=object)

r = -1
c = 0
for f in range(len(freqs)):
    if(f==0 or f==5):
        show_ylabels=True
        r += 1
        c = 0
    else: show_ylabels=False 
    if(f<5): 
        show_xlabels=False
    else: show_xlabels=True
    filename = intensity_pcolormesh(snaps[s], masss[s], scale, 
                              f, vmin_intensity_noratio, 
                              vmax_intensity_noratio,
                              saveloc=saveloc,
                              show_xlabels = show_xlabels, 
                              show_ylabels = show_ylabels, 
                              show_cbar=False)
    Inu12msun_files_4000au[r,c] = filename
    # print(r,c)
    c+=1

In [ ]:
cname = intensity_pcolormesh(snaps[s], years[s], scale=25000, f=f,
                          vmin=vmin_intensity_noratio, vmax=vmax_intensity_noratio,
                          show_xlabels=False, show_ylabels=False, 
                          show_cbar=True, vertical_cbar=True,
                                    saveloc=saveloc+'/cbars', cbar_pad=0.05)
cbimg = plot.save_cbar(cname, savename=saveloc+'/cbar_vertical_flooded.png', orientation='vertical')
# display(cbimg)
cbimg = cbimg.resize((int(1.5*cbimg.width), int(1.5*cbimg.height)))

allimage, details = plot.pil_image(Inu12msun_files_4000au)
allimage = plot.img_concat_right(allimage, cbimg) #, yy=cbar_yy)
display(allimage)
allimage.save(VICO_loc+'/figures/intensity_maps/Inu12Msun_flooded_4000au.png')

# 3 Cooling 25000 au 

In [ ]:
s=4
saveloc = VICO_loc+'/pillowfiles/intensities_12msun_fig'

nrows = 2
ncols = 5
scale = 25000
Inu12msun_files_cool_25000au = np.empty((nrows, ncols), dtype=object)

r = -1
c = 0
for f in range(len(freqs)):
    if(f==0 or f==5):
        show_ylabels=True
        r += 1
        c = 0
    else: show_ylabels=False 
    if(f<5): 
        show_xlabels=False
    else: show_xlabels=True
    filename = intensity_pcolormesh(snaps[s], masss[s], scale, 
                              f, vmin_intensity_noratio, 
                              vmax_intensity_noratio,
                              saveloc=saveloc,
                              show_xlabels = show_xlabels, 
                              show_ylabels = show_ylabels, 
                              show_cbar=False)
    Inu12msun_files_cool_25000au[r,c] = filename
    # print(r,c)
    c+=1

In [ ]:
cname = intensity_const_pcolormesh(snaps[s], years[s], False, 4000, f=f,
                          vmin=vmin_intensity_ratio, vmax=vmax_intensity_ratio,
                          show_xlabels=False, show_ylabels=False, 
                          show_cbar=True, vertical_cbar=True,
                                    saveloc=saveloc, cbar_pad=0.13)
cbimg = plot.save_cbar(cname, savename=saveloc+'/cbar_vertical_cooling.png', orientation='vertical')
cbimg = cbimg.resize((int(1.5*cbimg.width), int(1.5*cbimg.height)))

allimage, details = plot.pil_image(Inu12msun_files_cool_25000au)
allimage = plot.img_concat_right(allimage, cbimg) #, yy=cbar_yy)
display(allimage)
allimage.save(VICO_loc+'/figures/intensity_maps/Inu12Msun_cooling_25000au.png')

# 4 Flooded 25000 au 

In [ ]:
s=4
saveloc = VICO_loc+'/pillowfiles/intensities_12msun_fig'

nrows = 2
ncols = 5
scale = 4000
Inu12msun_files_cool_4000au = np.empty((nrows, ncols), dtype=object)

r = -1
c = 0
for f in range(len(freqs)):
    if(f==0 or f==5):
        show_ylabels=True
        r += 1
        c = 0
    else: show_ylabels=False 
    if(f<5): 
        show_xlabels=False
    else: show_xlabels=True
    filename = intensity_pcolormesh(snaps[s], masss[s], scale, 
                              f, vmin_intensity_noratio, 
                              vmax_intensity_noratio,
                              saveloc=saveloc,
                              show_xlabels = show_xlabels, 
                              show_ylabels = show_ylabels, 
                              show_cbar=False)
    Inu12msun_files_cool_4000au[r,c] = filename
    # print(r,c)
    c+=1

In [ ]:
cname = intensity_const_pcolormesh(snaps[s], years[s], False, 4000, f=f,
                          vmin=vmin_intensity_ratio, vmax=vmax_intensity_ratio,
                          show_xlabels=False, show_ylabels=False, 
                          show_cbar=True, vertical_cbar=True,
                                    saveloc=saveloc, cbar_pad=0.13)
cbimg = plot.save_cbar(cname, savename=saveloc+'/cbar_vertical_cooling.png', orientation='vertical')
cbimg = cbimg.resize((int(1.5*cbimg.width), int(1.5*cbimg.height)))

allimage, details = plot.pil_image(Inu12msun_files_cool_4000au)
allimage = plot.img_concat_right(allimage, cbimg) #, yy=cbar_yy)
display(allimage)
allimage.save(VICO_loc+'/figures/intensity_maps/Inu12Msun_cooling_4000au.png')